In [1]:
import os
import json
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import pylab as pl
from scipy.sparse import lil_matrix
%matplotlib inline

In [2]:
recipes = [f.path.split("/")[1] for f in os.scandir("recipe_data/") if f.is_dir() ]    
print(recipes[:10])
print("Nb of recipes: " + str(len(recipes)))

['229804', '245348', '25093', '14930', '257743', '255991', '236394', '13148', '18454', '215435']
Nb of recipes: 44069


In [29]:
def getRecipeNutrition(recipe_id):
    directory = 'recipe_data/' + recipe_id + "/"
    fname = directory + "nutrition.json"
    if os.path.isfile(fname) == False :
        return False
    
    with open('recipe_data/' + recipe_id + "/nutrition.json", 'r') as infile:
        nutrition = json.load(infile)['nutrition']
        
    return nutrition

In [32]:
def getRecipeFacts(recipe_id):
    nutrition = getRecipeNutrition(recipe_id)
    facts_list = ['sodium', 'saturatedFat', 'fat', 'sugars']
    facts = {}
    for fact in facts_list:
        if nutrition[fact]["unit"] == 'g':
            facts[fact] = nutrition[fact]["amount"]
        else:
            if nutrition[fact]["unit"] == 'mg':
                facts[fact] = nutrition[fact]["amount"] / 1000
            else:
                print("Can't get")
    return facts

In [43]:
def getRecipeHealthScore(recipe_id):
    facts = getRecipeFacts(recipe_id)
    servingWeight = 300
    score = 0
    
    if facts['fat'] < servingWeight/100 * 3:
        score += 1
    else:
        if facts['fat'] < servingWeight/100 * 17.5:
            score += 2
        else:
            score += 3
            
    if facts['saturatedFat'] < servingWeight/100 * 1.5:
        score += 1
    else:
        if facts['saturatedFat'] < servingWeight/100 * 5:
            score += 2
        else:
            score += 3    
            
    if facts['sugars'] < servingWeight/100 * 5:
        score += 1
    else:
        if facts['sugars'] < servingWeight/100 * 22.5:
            score += 2
        else:
            score += 3
            
    if facts['sodium'] < servingWeight/100 * 0.3:
        score += 1
    else:
        if facts['sodium'] < servingWeight/100 * 1.5:
            score += 2
        else:
            score += 3
    
    return score

In [44]:
getRecipeHealthScore("15184")

5